# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [20]:
# Write your code below.
%load_ext dotenv
%dotenv 

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [21]:
import os
print(os.getenv("PRICE_DATA"))

../../05_src/data/prices/


In [22]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [68]:
import os
import sys
from glob import glob

# Write your code below.
# Append custom source directory to sys.path
sys.path.append(os.getenv('SRC_DIR'))

# Import custom logger
from utils.logger import get_logger
_logs = get_logger(__name__)

# Load the PRICE_DATA environment variable
price_data_dir = os.getenv('PRICE_DATA')


    
# Use glob to find all Parquet files in the PRICE_DATA directory
parquet_files = glob(os.path.join(price_data_dir, "**/*.parquet"),recursive=True)


# Log the number of files found
_logs.info(f"Found {len(parquet_files)} parquet files in {price_data_dir}")

ddf = dd.read_parquet(parquet_files)
ddf



2025-06-07 12:15:09,561, 3728842318.py, 23, INFO, Found 2045 parquet files in ../../05_src/data/prices/


,Date,Open,High,Low,Close,Adj Close,Volume,source,ticker,Year
npartitions=2045,,,,,,,,,,
,datetime64[ns],float64,float64,float64,float64,float64,float64,object,object,int32
,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [ ]:
# Write your code below.
import dask.dataframe as dd

In [ ]:
# Function to add features per group (per ticker)
#def add_features(ddf):
ddf = ddf.sort_values("Date")  # Make sure it's sorted by time
ddf["Close_lag_1"] = ddf.groupby("ticker")["Close"].shift(1)
ddf["Adj_Close_lag_1"] = ddf.groupby("ticker")["Adj Close"].shift(1)
ddf["returns"] = (ddf["Close"] / ddf["Close_lag_1"]) - 1
ddf["hi_lo_range"] = ddf["High"] - ddf["Low"]
    

C:\Users\sunny\AppData\Local\Temp\ipykernel_4864\186450895.py:4: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  ddf["Close_lag_1"] = ddf.groupby("ticker")["Close"].shift(1)
C:\Users\sunny\AppData\Local\Temp\ipykernel_4864\186450895.py:5: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  ddf["Adj_Close_lag_1"] = ddf.groupby("ticker")["Adj Close"].shift(1)


In [ ]:
#return ddf:
ddf

,Date,Open,High,Low,Close,Adj Close,Volume,source,ticker,Year,Close_lag_1,Adj_Close_lag_1,returns,hi_lo_range
npartitions=100,,,,,,,,,,,,,,
,datetime64[ns],float64,float64,float64,float64,float64,float64,object,object,int32,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [66]:
pdf = ddf.compute()
pdf

pdf.groupby("ticker").apply(
    lambda x : x.assign(moving_avg = x["returns"].rolling(10).mean())
)

C:\Users\sunny\AppData\Local\Temp\ipykernel_4864\1655953788.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  pdf.groupby("ticker").apply(


Date       Open       High        Low      Close  \
ticker                                                                 
ACN    137602 2002-09-13  16.000000  16.090000  15.250000  15.750000   
       138017 2004-05-07  23.750000  24.080000  23.580000  23.690001   
       138038 2004-06-08  25.350000  25.620001  25.190001  25.480000   
       138044 2004-06-17  26.240000  27.040001  25.940001  26.790001   
       139285 2009-05-21  29.959999  30.000000  29.629999  29.879999   
...                  ...        ...        ...        ...        ...   
ALL    74531  2008-04-14  48.090000  48.119999  47.599998  47.849998   
       75435  2011-11-10  26.139999  26.389999  26.020000  26.200001   
       75478  2012-01-13  28.770000  29.040001  28.549999  28.980000   
       72601  2000-08-07  29.562500  29.562500  28.312500  28.562500   
       73959  2006-01-03  54.389999  54.849998  53.750000  54.730000   

               Adj Close      Volume   source ticker  Year  Close_lag_1  \
ticker                                                                    
ACN    137602  11.840421   2727700.0  ACN.csv    ACN  2002    16.299999   
       138017  17.809496   4080000.0  ACN.csv    ACN  2004    24.129999   
       138038  19.155170   2031500.0  ACN.csv    ACN  2004    25.170000   
       138044  20.139994  10099800.0  ACN.csv    ACN  2004    25.309999   
       139285  23.599159   2766400.0  ACN.csv    ACN  2009    30.340000   
...                  ...         ...      ...    ...   ...          ...   
ALL    74531   35.826778   2594800.0  ALL.csv    ALL  2008    48.200001   
       75435   22.063234   4610200.0  ALL.csv    ALL  2011    25.850000   
       75478   24.613558   2911100.0  ALL.csv    ALL  2012    29.070000   
       72601   17.770233   2677000.0  ALL.csv    ALL  2000    29.500000   
       73959   38.596119   3192200.0  ALL.csv    ALL  2006    54.070000   

               Adj_Close_lag_1   returns  hi_lo_range  moving_avg  
ticker                                                             
ACN    137602        12.253895 -0.033742     0.840000         NaN  
       138017        18.140272 -0.018235     0.500000         NaN  
       138038        18.922119  0.012316     0.430000         NaN  
       138044        19.027370  0.058475     1.100000         NaN  
       139285        23.962473 -0.015162     0.370001         NaN  
...                        ...       ...          ...         ...  
ALL    74531         36.088837 -0.007261     0.520000    0.002345  
       75435         21.768488  0.013540     0.369999    0.002663  
       75478         24.690002 -0.003096     0.490002    0.001757  
       72601         18.353498 -0.031780     1.250000   -0.001682  
       73959         38.130669  0.012206     1.099998   -0.003218  

[11883 rows x 15 columns]

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

Not necessarily but for small to medium datasets fitting in memory, pandas conversion is convenient and efficient.

For large datasets that exceed memory, Dask is a better choice for calculating moving averages efficiently and scalably.


(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.